# Лабораторная работа 2 (Linear Model)

# Импорт всех необходимых модулей

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    LabelEncoder,
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
)
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet


def regression_cross_validate(model_cls, X, y, n_folds: int = 5, **model_kwargs):
    X = np.asarray(X)
    y = np.asarray(y)

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    mae_values = []
    mse_values = []
    r2_values = []

    for fold_idx, (idx_train, idx_valid) in enumerate(kf.split(X), start=1):
        X_train, X_valid = X[idx_train], X[idx_valid]
        y_train, y_valid = y[idx_train], y[idx_valid]

        model = model_cls(**model_kwargs)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_valid)

        mae_values.append(mean_absolute_error(y_valid, y_pred))
        mse_values.append(mean_squared_error(y_valid, y_pred))
        r2_values.append(r2_score(y_valid, y_pred))

    metrics_avg = {
        "MAE": float(np.mean(mae_values)),
        "MSE": float(np.mean(mse_values)),
        "R2": float(np.mean(r2_values)),
    }

    metrics_std = {
        "MAE": float(np.std(mae_values)),
        "MSE": float(np.std(mse_values)),
        "R2": float(np.std(r2_values)),
    }

    return metrics_avg, metrics_std


def classification_cross_validate(model_cls, X, y, n_folds: int = 5, **model_kwargs):
    X = np.asarray(X)
    y = np.asarray(y)

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    acc_list = []
    prec_list = []
    rec_list = []
    f1_list = []

    for fold_idx, (idx_train, idx_valid) in enumerate(kf.split(X), start=1):
        X_train, X_valid = X[idx_train], X[idx_valid]
        y_train, y_valid = y[idx_train], y[idx_valid]

        model = model_cls(**model_kwargs)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_valid)

        acc_list.append(accuracy_score(y_valid, y_pred))
        prec_list.append(precision_score(y_valid, y_pred, average="weighted", zero_division=0))
        rec_list.append(recall_score(y_valid, y_pred, average="weighted", zero_division=0))
        f1_list.append(f1_score(y_valid, y_pred, average="weighted", zero_division=0))

    metrics_avg = {
        "Accuracy": float(np.mean(acc_list)),
        "Precision": float(np.mean(prec_list)),
        "Recall": float(np.mean(rec_list)),
        "F1-score": float(np.mean(f1_list)),
    }

    metrics_std = {
        "Accuracy": float(np.std(acc_list)),
        "Precision": float(np.std(prec_list)),
        "Recall": float(np.std(rec_list)),
        "F1-score": float(np.std(f1_list)),
    }

    return metrics_avg, metrics_std


# Часть 1. Регрессия
Данные о зарплатах загружаются в DataFrame, и выводятся первые строки, чтобы убедиться, что файл прочитан корректно.

In [19]:
salary_df = pd.read_csv("data/Salary Data.csv")
print("Размер датасета Salary_Data:", salary_df.shape)

salary_df.head()

Размер датасета Salary_Data: (375, 6)


,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


# Подготовка данных для регрессии

Сначала из датасета убирается столбец `Job Title`: он содержит много уникальных значений и в таком виде мало помогает линейной модели, только раздувает пространство признаков

Категориальные признаки (`Gender`, `Education Level`) переводятся в числовой вид с помощью `OrdinalEncoder`, чтобы их можно было использовать в линейной регрессии и в собственной реализации модели

После этого все пропуски в данных заполняются наиболее частыми значениями (`SimpleImputer` с стратегией `"most_frequent"`). Такой шаг нужен, потому что большинство моделей из `sklearn`, а также наши собственные реализации, не умеют работать с `NaN` и ожидают полностью числовую матрицу признаков без пропусков


In [20]:
reg_df = salary_df.copy()
reg_df = reg_df.dropna(subset=["Salary"])

if "Job Title" in reg_df.columns:
    reg_df = reg_df.drop(columns=["Job Title"])

X_reg = reg_df.drop(columns=["Salary"])
y_reg = reg_df["Salary"]

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

reg_categorical_cols = ["Gender", "Education Level"]
reg_numeric_cols = [col for col in X_reg.columns if col not in reg_categorical_cols]

reg_ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

X_reg_train_base = X_reg_train.copy()
X_reg_test_base = X_reg_test.copy()

X_reg_train_base[reg_categorical_cols] = reg_ordinal_encoder.fit_transform(
    X_reg_train_base[reg_categorical_cols]
)
X_reg_test_base[reg_categorical_cols] = reg_ordinal_encoder.transform(
    X_reg_test_base[reg_categorical_cols]
)

reg_imputer = SimpleImputer(strategy="most_frequent")

X_reg_train_base = pd.DataFrame(
    reg_imputer.fit_transform(X_reg_train_base),
    columns=X_reg_train_base.columns
)
X_reg_test_base = pd.DataFrame(
    reg_imputer.transform(X_reg_test_base),
    columns=X_reg_test_base.columns
)

print("NaN в X_train_base:", X_reg_train_base.isna().sum().sum())
print("NaN в X_test_base:", X_reg_test_base.isna().sum().sum())


NaN в X_train_base: 0
NaN в X_test_base: 0


Сначала через кросс-валидацию на обучающей выборке считаются средние значения метрик, чтобы получить устойчивую оценку качества. Затем та же модель обучается на всём train-наборе, тестируется на отложенных данных и выводятся метрики на test-выборке для сравнения с последующими улучшенными вариантами

In [21]:
reg_cv_mean, reg_cv_std = regression_cross_validate(
    LinearRegression,
    X_reg_train_base.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
)

linreg_baseline = LinearRegression()
linreg_baseline.fit(X_reg_train_base, y_reg_train)

y_reg_pred_test = linreg_baseline.predict(X_reg_test_base)

test_mse = mean_squared_error(y_reg_test, y_reg_pred_test)
test_mae = mean_absolute_error(y_reg_test, y_reg_pred_test)
test_r2 = r2_score(y_reg_test, y_reg_pred_test)

print("=== Регрессия: бейзлайн (train CV) ===")
print(f"MSE_mean: {reg_cv_mean['MSE']:.2f}")
print(f"MAE_mean: {reg_cv_mean['MAE']:.2f}")
print(f"R2_mean:  {reg_cv_mean['R2']:.3f}")

print("\nРегрессия - бейзлайн (test)")
print("---------------------------")
print(f"MSE: {test_mse:.2f}")
print(f"MAE: {test_mae:.2f}")
print(f"R^2: {test_r2:.3f}")


=== Регрессия: бейзлайн (train CV) ===
MSE_mean: 239121094.97
MAE_mean: 11692.15
R2_mean:  0.896

Регрессия - бейзлайн (test)
---------------------------
MSE: 232595028.53
MAE: 10613.91
R^2: 0.903


# Улучшенный бейзлайн для регрессии

Дальше пробуем усилить модель за счёт более аккуратной обработки признаков. Для категориальных столбцов (`Gender`, `Education Level`) вместо простого численного кодирования используется `OneHotEncoder`, который разворачивает их в набор бинарных признаков . Числовые признаки (`Age`, `Years of Experience`) дополнительно приводятся к одному масштабу с помощью `StandardScaler`, чтобы ни один из них не доминировал за счёт больших значений: на выходе получаем обновлённую обучающую и тестовую матрицы (`X_reg_train_scaled`, `X_reg_test_scaled`), которые дальше используются в улучшенном варианте модели.


In [22]:
reg_onehot = OneHotEncoder(
    sparse_output=False,
    drop="first",
    handle_unknown="ignore",
)

reg_categorical_cols = ["Gender", "Education Level"]
reg_numeric_cols = [col for col in X_reg.columns if col not in reg_categorical_cols]

train_cat_ohe = reg_onehot.fit_transform(X_reg_train[reg_categorical_cols])
test_cat_ohe = reg_onehot.transform(X_reg_test[reg_categorical_cols])

ohe_reg_cols = reg_onehot.get_feature_names_out(reg_categorical_cols)

X_reg_train_ohe = pd.concat(
    [
        X_reg_train[reg_numeric_cols].reset_index(drop=True),
        pd.DataFrame(train_cat_ohe, columns=ohe_reg_cols),
    ],
    axis=1,
)

X_reg_test_ohe = pd.concat(
    [
        X_reg_test[reg_numeric_cols].reset_index(drop=True),
        pd.DataFrame(test_cat_ohe, columns=ohe_reg_cols),
    ],
    axis=1,
)


reg_scaler = StandardScaler()
X_reg_train_scaled = X_reg_train_ohe.copy()
X_reg_test_scaled = X_reg_test_ohe.copy()

X_reg_train_scaled[reg_numeric_cols] = reg_scaler.fit_transform(
    X_reg_train_ohe[reg_numeric_cols]
)
X_reg_test_scaled[reg_numeric_cols] = reg_scaler.transform(
    X_reg_test_ohe[reg_numeric_cols]
)


на кросс-валидации считается качество модели ElasticNet, обученной на one-hot + отмасштабированных признаках. Затем та же модель дообучается на всей обучающей выборке, тестируется на отложенных данных, и по MSE/MAE/R^2 можно сравнить, насколько регуляризация и улучшенный препроцессинг помогают по сравнению с простой линейной регрессией

In [23]:
reg_enet_cv_mean, reg_enet_cv_std = regression_cross_validate(
    ElasticNet,
    X_reg_train_scaled.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    alpha=0.01,
    l1_ratio=0.7,
)

print("=== Регрессия: улучшенный бейзлайн (train CV) ===")
print(f"MSE_mean: {reg_enet_cv_mean['MSE']:.2f}")
print(f"MAE_mean: {reg_enet_cv_mean['MAE']:.2f}")
print(f"R2_mean:  {reg_enet_cv_mean['R2']:.3f}")

enet_model = ElasticNet(alpha=0.01, l1_ratio=0.7)
enet_model.fit(X_reg_train_scaled, y_reg_train)

y_reg_pred_enet = enet_model.predict(X_reg_test_scaled)

mse_enet = mean_squared_error(y_reg_test, y_reg_pred_enet)
mae_enet = mean_absolute_error(y_reg_test, y_reg_pred_enet)
r2_enet = r2_score(y_reg_test, y_reg_pred_enet)

print("\nРегрессия - улучшенный бейзлайн (test)")
print("--------------------------------------")
print(f"MSE: {mse_enet:.2f}")
print(f"MAE: {mae_enet:.2f}")
print(f"R^2: {r2_enet:.3f}")


=== Регрессия: улучшенный бейзлайн (train CV) ===
MSE_mean: 233219582.60
MAE_mean: 11262.21
R2_mean:  0.898

Регрессия - улучшенный бейзлайн (test)
--------------------------------------
MSE: 227094653.11
MAE: 10488.54
R^2: 0.905


# Реализация своего класса

В этом классе реализована упрощённая версия модели с регуляризацией ElasticNet: сначала к признакам добавляется столбец единиц, и параметры линейной регрессии оцениваются через нормальное уравнение. Затем к найденным весам добавляется смесь L1- и L2-штрафов, после чего регуляризованные коэффициенты и интерсепт сохраняются в `self.coef_` и `self.intercept_`, а метод `predict` использует их для вычисления линейного прогноза


In [24]:
class MyElasticNet:
    def __init__(self, alpha=1.0, l1_ratio=0.5, max_iter=1000, tol=1e-4):
        self.alpha = alpha
        self.l1_ratio = l1_ratio

        self.max_iter = max_iter
        self.tol = tol

        self.coef_ = None      
        self.intercept_ = None 

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        design_matrix = np.c_[np.ones(X.shape[0]), X]

        theta = np.linalg.pinv(design_matrix.T @ design_matrix) @ design_matrix.T @ y

        intercept = theta[0]
        weights = theta[1:]

        l1_penalty = self.alpha * self.l1_ratio * np.sign(weights)
        l2_penalty = self.alpha * (1.0 - self.l1_ratio) * weights

        weights = weights - (l1_penalty + l2_penalty)

        self.intercept_ = intercept
        self.coef_ = weights

        return self

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        return X @ self.coef_ + self.intercept_


Cначала через кросс-валидацию без регуляризации (alpha=0, l1_ratio=0) оцениваются средние метрики на обучающей выборке. Затем та же модель обучается на базовых признаках и проверяется на тесте

In [25]:
reg_my_base_mean, reg_my_base_std = regression_cross_validate(
    MyElasticNet,
    X_reg_train_base.to_numpy(), 
    y_reg_train.to_numpy(),
    n_folds=5,
    alpha=0.0,
    l1_ratio=0.0,
)

print("=== Регрессия: собственная модель (train CV, без улучшений) ===")
print(f"MSE_mean: {reg_my_base_mean['MSE']:.2f}")
print(f"MAE_mean: {reg_my_base_mean['MAE']:.2f}")
print(f"R2_mean:  {reg_my_base_mean['R2']:.3f}")

my_enet_base = MyElasticNet(alpha=0.0, l1_ratio=0.0)
my_enet_base.fit(X_reg_train_base, y_reg_train)

y_reg_pred_my_base = my_enet_base.predict(X_reg_test_base)

mse_my_base = mean_squared_error(y_reg_test, y_reg_pred_my_base)
mae_my_base = mean_absolute_error(y_reg_test, y_reg_pred_my_base)
r2_my_base = r2_score(y_reg_test, y_reg_pred_my_base)

print("\nРегрессия - собственная реализация (test, без улучшений)")
print("--------------------------------------------------------")
print(f"MSE: {mse_my_base:.2f}")
print(f"MAE: {mae_my_base:.2f}")
print(f"R^2: {r2_my_base:.3f}")


=== Регрессия: собственная модель (train CV, без улучшений) ===
MSE_mean: 239121094.97
MAE_mean: 11692.15
R2_mean:  0.896

Регрессия - собственная реализация (test, без улучшений)
--------------------------------------------------------
MSE: 232595028.53
MAE: 10613.91
R^2: 0.903


MyElasticNet обучается на one-hot + отмасштабированных признаках с ненулевой регуляризацией, а качество сначала измеряется по кросс-валидации

In [26]:
reg_my_imp_mean, reg_my_imp_std = regression_cross_validate(
    MyElasticNet,
    X_reg_train_scaled.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    alpha=0.01,
    l1_ratio=0.7,
)

print("=== Регрессия: собственная реализация (train CV, с улучшениями) ===")
print(f"MSE_mean: {reg_my_imp_mean['MSE']:.2f}")
print(f"MAE_mean: {reg_my_imp_mean['MAE']:.2f}")
print(f"R2_mean:  {reg_my_imp_mean['R2']:.3f}")

my_enet_improved = MyElasticNet(alpha=0.01, l1_ratio=0.7)
my_enet_improved.fit(X_reg_train_scaled, y_reg_train)

y_reg_pred_my_imp = my_enet_improved.predict(X_reg_test_scaled)

mse_my_imp = mean_squared_error(y_reg_test, y_reg_pred_my_imp)
mae_my_imp = mean_absolute_error(y_reg_test, y_reg_pred_my_imp)
r2_my_imp = r2_score(y_reg_test, y_reg_pred_my_imp)

print("\nРегрессия — собственная реализация (test, с улучшениями)")
print("--------------------------------------------------------")
print(f"MSE: {mse_my_imp:.2f}")
print(f"MAE: {mae_my_imp:.2f}")
print(f"R^2: {r2_my_imp:.3f}")


=== Регрессия: собственная реализация (train CV, с улучшениями) ===
MSE_mean: 233135666.22
MAE_mean: 11303.02
R2_mean:  0.898

Регрессия — собственная реализация (test, с улучшениями)
--------------------------------------------------------
MSE: 232347419.73
MAE: 10616.83
R^2: 0.903


# Часть 2. Классификация

# Обработка данных

In [27]:
stud_df = pd.read_csv('data/Student Depression Dataset.csv')
stud_df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


# Подготовка данных для классификации

Сначала из датасета копируется таблица и разделяются признаки и целевая переменная: столбец `Depression` берётся как метка класса, а `id` удаляется как технический идентификатор. Далее выборка разбивается на обучающую и тестовую части с `stratify=y_clf`, чтобы сохранить исходное распределение классов. Отдельно задаются списки категориальных и числовых признаков, после чего все пропуски в данных заполняются наиболее частыми значениями с помощью `SimpleImputer`, чтобы логистическая регрессия и собственная модель могли работать с полной матрицей без `NaN`.


In [28]:
clf_df = stud_df.copy()

X_clf = clf_df.drop(columns=["Depression", "id"])
y_clf = clf_df["Depression"]

X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf,
    y_clf,
    test_size=0.2,
    random_state=42,
    stratify=y_clf,
)

clf_categorical_cols = [
    "Gender",
    "City",
    "Profession",
    "Sleep Duration",
    "Dietary Habits",
    "Degree",
    "Have you ever had suicidal thoughts ?",
    "Family History of Mental Illness",
]

clf_numeric_cols = [
    "Age",
    "Academic Pressure",
    "Work Pressure",
    "CGPA",
    "Study Satisfaction",
    "Job Satisfaction",
    "Work/Study Hours",
    "Financial Stress",
]

clf_imputer = SimpleImputer(strategy="most_frequent")

X_clf_train_imp = pd.DataFrame(
    clf_imputer.fit_transform(X_clf_train),
    columns=X_clf_train.columns,
)
X_clf_test_imp = pd.DataFrame(
    clf_imputer.transform(X_clf_test),
    columns=X_clf_test.columns,
)

X_clf_train_imp.head()

,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,Male,18.0,Jaipur,Student,4.0,0.0,6.02,1.0,0.0,7-8 hours,Moderate,Class 12,Yes,3.0,5.0,No
1,Male,25.0,Vadodara,Student,3.0,0.0,6.37,2.0,0.0,7-8 hours,Moderate,B.Arch,No,9.0,1.0,Yes
2,Male,30.0,Ahmedabad,Student,3.0,0.0,9.24,2.0,0.0,7-8 hours,Unhealthy,M.Ed,Yes,5.0,5.0,Yes
3,Male,34.0,Bhopal,Student,3.0,0.0,7.37,5.0,0.0,7-8 hours,Moderate,B.Com,Yes,12.0,3.0,No
4,Male,25.0,Patna,Student,3.0,0.0,7.47,4.0,0.0,5-6 hours,Unhealthy,B.Com,No,11.0,5.0,No


# Построение бейзлайна 

Кодируем категориальные признаки через `OrdinalEncoder`, убеждается, что пропусков больше нет, и на этих данных строит бейзлайновую логистическую регрессию: сначала считает средние метрики по кросс-валидации, затем оценивает качество на тестовой выборке


In [29]:
clf_ordinal_encoder = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=99,
)

X_clf_train_base = X_clf_train_imp.copy()
X_clf_test_base = X_clf_test_imp.copy()

X_clf_train_base[clf_categorical_cols] = clf_ordinal_encoder.fit_transform(
    X_clf_train_base[clf_categorical_cols]
)
X_clf_test_base[clf_categorical_cols] = clf_ordinal_encoder.transform(
    X_clf_test_base[clf_categorical_cols]
)

print("NaN в X_clf_train_base:", X_clf_train_base.isna().sum().sum())
print("NaN в X_clf_test_base:", X_clf_test_base.isna().sum().sum())

clf_cv_mean_base, clf_cv_std_base = classification_cross_validate(
    LogisticRegression,
    X_clf_train_base.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    max_iter=1000,
)

print("\n=== Классификация: бейзлайн (train CV) ===")
print(f"Accuracy_mean:  {clf_cv_mean_base['Accuracy']:.3f}")
print(f"Precision_mean:{clf_cv_mean_base['Precision']:.3f}")
print(f"Recall_mean:   {clf_cv_mean_base['Recall']:.3f}")
print(f"F1_mean:       {clf_cv_mean_base['F1-score']:.3f}")


clf_baseline_model = LogisticRegression(max_iter=1000)
clf_baseline_model.fit(X_clf_train_base, y_clf_train)

y_clf_pred_base = clf_baseline_model.predict(X_clf_test_base)

acc_base = accuracy_score(y_clf_test, y_clf_pred_base)
prec_base = precision_score(y_clf_test, y_clf_pred_base, average="weighted", zero_division=0)
rec_base = recall_score(y_clf_test, y_clf_pred_base, average="weighted", zero_division=0)
f1_base = f1_score(y_clf_test, y_clf_pred_base, average="weighted", zero_division=0)

print("\nКлассификация - бейзлайн (test)")
print("--------------------------------")
print(f"1. Accuracy:  {acc_base:.2%}")
print(f"2. Precision: {prec_base:.2%}")
print(f"3. Recall:    {rec_base:.2%}")
print(f"4. F1-score:  {f1_base:.2%}")


NaN в X_clf_train_base: 0
NaN в X_clf_test_base: 0

=== Классификация: бейзлайн (train CV) ===
Accuracy_mean:  0.847
Precision_mean:0.846
Recall_mean:   0.847
F1_mean:       0.846

Классификация - бейзлайн (test)
--------------------------------
1. Accuracy:  84.29%
2. Precision: 84.23%
3. Recall:    84.29%
4. F1-score:  84.24%


# Формулировка гипотез

Сформулируем несколько идей для улучшения классификатора: закодировать категориальные признаки через `OneHotEncoder`, чтобы не задавать им искусственный порядок, и нормировать числовые столбцы, чтобы все признаки были в сопоставимом масштабе. В этом блоке как раз выполняются one-hot кодирование категориальных признаков и масштабирование числовых, после чего формируются обновлённые матрицы признаков `X_clf_train_upd` и `X_clf_test_upd` для улучшенной логистической регрессии


In [30]:
clf_onehot = OneHotEncoder(
    sparse_output=False,
    drop="first",
    handle_unknown="ignore",
)

train_cat_ohe = clf_onehot.fit_transform(X_clf_train_imp[clf_categorical_cols])
test_cat_ohe = clf_onehot.transform(X_clf_test_imp[clf_categorical_cols])

ohe_clf_cols = clf_onehot.get_feature_names_out(clf_categorical_cols)

X_clf_train_ohe = pd.concat(
    [
        X_clf_train_imp[clf_numeric_cols].reset_index(drop=True),
        pd.DataFrame(train_cat_ohe, columns=ohe_clf_cols),
    ],
    axis=1,
)

X_clf_test_ohe = pd.concat(
    [
        X_clf_test_imp[clf_numeric_cols].reset_index(drop=True),
        pd.DataFrame(test_cat_ohe, columns=ohe_clf_cols),
    ],
    axis=1,
)

clf_scaler = StandardScaler()
X_clf_train_upd = X_clf_train_ohe.copy()
X_clf_test_upd = X_clf_test_ohe.copy()

X_clf_train_upd[clf_numeric_cols] = clf_scaler.fit_transform(
    X_clf_train_ohe[clf_numeric_cols]
)
X_clf_test_upd[clf_numeric_cols] = clf_scaler.transform(
    X_clf_test_ohe[clf_numeric_cols]
)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Оцениваем улучшенную логистическую регрессию

In [31]:
clf_cv_mean_imp, clf_cv_std_imp = classification_cross_validate(
    LogisticRegression,
    X_clf_train_upd.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    C=0.1,
    max_iter=1000,
)

print("=== Классификация: улучшенный бейзлайн (train CV) ===")
print(f"Accuracy_mean:  {clf_cv_mean_imp['Accuracy']:.3f}")
print(f"Precision_mean:{clf_cv_mean_imp['Precision']:.3f}")
print(f"Recall_mean:   {clf_cv_mean_imp['Recall']:.3f}")
print(f"F1_mean:       {clf_cv_mean_imp['F1-score']:.3f}")

clf_improved_model = LogisticRegression(C=0.1, max_iter=1000)
clf_improved_model.fit(X_clf_train_upd, y_clf_train)

y_clf_pred_improved = clf_improved_model.predict(X_clf_test_upd)

acc_imp = accuracy_score(y_clf_test, y_clf_pred_improved)
prec_imp = precision_score(y_clf_test, y_clf_pred_improved, average="weighted", zero_division=0)
rec_imp = recall_score(y_clf_test, y_clf_pred_improved, average="weighted", zero_division=0)
f1_imp = f1_score(y_clf_test, y_clf_pred_improved, average="weighted", zero_division=0)

print("\nКлассификация - улучшенный бейзлайн (test)")
print("------------------------------------------")
print(f"1. Accuracy:  {acc_imp:.2%}")
print(f"2. Precision: {prec_imp:.2%}")
print(f"3. Recall:    {rec_imp:.2%}")
print(f"4. F1-score:  {f1_imp:.2%}")


=== Классификация: улучшенный бейзлайн (train CV) ===
Accuracy_mean:  0.849
Precision_mean:0.849
Recall_mean:   0.849
F1_mean:       0.849

Классификация - улучшенный бейзлайн (test)
------------------------------------------
1. Accuracy:  84.50%
2. Precision: 84.45%
3. Recall:    84.50%
4. F1-score:  84.46%


# Реализация своего класса

Собственная реализация логистической регрессии: в fit модель обучается методом градиентного спуска с возможной L2-регуляризацией через параметр C, а в predict считаются вероятности через сигмоиду и возвращаются бинарные метки по порогу 0.5.

In [32]:
class MyLogisticRegression:

    def __init__(self, C=0.0, learning_rate=0.01, n_iterations=1000):
        self.C = C
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations

        self.weights: np.ndarray | None = None
        self.bias: float | None = None

    @staticmethod
    def _sigmoid(z):
        return 1.0 / (1.0 + np.exp(-z))

    def fit(self, X, y):

        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        n_samples, n_features = X.shape

        self.weights = np.zeros(n_features, dtype=float)
        self.bias = 0.0

        for _ in range(self.n_iterations):
            logits = X @ self.weights + self.bias
            probs = self._sigmoid(logits)

            grad_w = (1.0 / n_samples) * (X.T @ (probs - y))
            grad_b = (1.0 / n_samples) * np.sum(probs - y)

            if self.C != 0:
                grad_w += (1.0 / (self.C * n_samples)) * self.weights

            self.weights -= self.learning_rate * grad_w
            self.bias -= self.learning_rate * grad_b

        return self

    def predict(self, X):

        X = np.asarray(X, dtype=float)
        logits = X @ self.weights + self.bias
        probs = self._sigmoid(logits)
        return (probs >= 0.5).astype(int)


In [33]:
clf_my_base_mean, clf_my_base_std = classification_cross_validate(
    MyLogisticRegression,
    X_clf_train_base.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    n_iterations=1000,
)

print("=== Классификация: собственная модель (train CV, без улучшений) ===")
print(f"Accuracy_mean:  {clf_my_base_mean['Accuracy']:.3f}")
print(f"Precision_mean:{clf_my_base_mean['Precision']:.3f}")
print(f"Recall_mean:   {clf_my_base_mean['Recall']:.3f}")
print(f"F1_mean:       {clf_my_base_mean['F1-score']:.3f}")

my_logreg_base = MyLogisticRegression(n_iterations=1000)
my_logreg_base.fit(X_clf_train_base.to_numpy(), y_clf_train.to_numpy())

y_clf_pred_my_base = my_logreg_base.predict(X_clf_test_base.to_numpy())

acc_my_base = accuracy_score(y_clf_test, y_clf_pred_my_base)
prec_my_base = precision_score(y_clf_test, y_clf_pred_my_base, average="weighted", zero_division=0)
rec_my_base = recall_score(y_clf_test, y_clf_pred_my_base, average="weighted", zero_division=0)
f1_my_base = f1_score(y_clf_test, y_clf_pred_my_base, average="weighted", zero_division=0)

print("\nКлассификация - собственная реализация (test, без улучшений)")
print("------------------------------------------------------------")
print(f"1. Accuracy:  {acc_my_base:.2%}")
print(f"2. Precision: {prec_my_base:.2%}")
print(f"3. Recall:    {rec_my_base:.2%}")
print(f"4. F1-score:  {f1_my_base:.2%}")


=== Классификация: собственная модель (train CV, без улучшений) ===
Accuracy_mean:  0.769
Precision_mean:0.804
Recall_mean:   0.769
F1_mean:       0.770

Классификация - собственная реализация (test, без улучшений)
------------------------------------------------------------
1. Accuracy:  76.76%
2. Precision: 80.38%
3. Recall:    76.76%
4. F1-score:  76.84%


по кросс-валидации считаются средние метрики на one-hot + масштабированных признаках с регуляризацией C=0.1, затем та же модель обучается на всём train-наборе и оценивается на тесте, где выводятся итоговые Accuracy, Precision, Recall и F1-score.

In [34]:
clf_my_imp_mean, clf_my_imp_std = classification_cross_validate(
    MyLogisticRegression,
    X_clf_train_upd.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    C=0.1,
    n_iterations=1000,
)

print("=== Классификация: собственная модель (train CV, с улучшениями) ===")
print(f"Accuracy_mean:  {clf_my_imp_mean['Accuracy']:.3f}")
print(f"Precision_mean:{clf_my_imp_mean['Precision']:.3f}")
print(f"Recall_mean:   {clf_my_imp_mean['Recall']:.3f}")
print(f"F1_mean:       {clf_my_imp_mean['F1-score']:.3f}")

my_logreg_improved = MyLogisticRegression(C=0.1, n_iterations=1000)
my_logreg_improved.fit(X_clf_train_upd.to_numpy(), y_clf_train.to_numpy())

y_clf_pred_my_imp = my_logreg_improved.predict(X_clf_test_upd.to_numpy())

acc_my_imp = accuracy_score(y_clf_test, y_clf_pred_my_imp)
prec_my_imp = precision_score(y_clf_test, y_clf_pred_my_imp, average="weighted", zero_division=0)
rec_my_imp = recall_score(y_clf_test, y_clf_pred_my_imp, average="weighted", zero_division=0)
f1_my_imp = f1_score(y_clf_test, y_clf_pred_my_imp, average="weighted", zero_division=0)

print("\nКлассификация - собственная реализация (test, с улучшениями)")
print("------------------------------------------------------------")
print(f"1. Accuracy:  {acc_my_imp:.2%}")
print(f"2. Precision: {prec_my_imp:.2%}")
print(f"3. Recall:    {rec_my_imp:.2%}")
print(f"4. F1-score:  {f1_my_imp:.2%}")


=== Классификация: собственная модель (train CV, с улучшениями) ===
Accuracy_mean:  0.830
Precision_mean:0.830
Recall_mean:   0.830
F1_mean:       0.828

Классификация - собственная реализация (test, с улучшениями)
------------------------------------------------------------
1. Accuracy:  82.31%
2. Precision: 82.35%
3. Recall:    82.31%
4. F1-score:  82.10%


## Заключение

Для задачи регрессии линейные модели показывают достаточно высокое качество уже в базовой постановке: `R^2` около **0.90** означает, что модель объясняет примерно 90 % вариации зарплаты. Переход к one-hot кодированию категориальных признаков, масштабированию числовых и использованию `ElasticNet` даёт небольшое, но стабильное улучшение метрик на тесте, однако прирост не радикален - что ожидаемо для относительно небольшого и простой структуры датасета.

Собственная реализация `MyElasticNet` по качеству практически не отличается от моделей из `sklearn`: без улучшений она совпадает с линейной регрессией, а с регуляризацией и улучшенным препроцессингом держится на сопоставимом уровне `R^2`, хотя по MSE/MAE чуть уступает эталонной `ElasticNet`.

В классификационной задаче логистическая регрессия уже в бейзлайне даёт около **84 %** по всем основным метрикам. Улучшенный вариант (one-hot кодирование + масштабирование + регуляризация) добавляет ещё ~0.2 процентных пункта. Собственная реализация логистической регрессии заметно выигрывает от улучшенного препроцессинга: с сырыми признаками она ощутимо хуже sklearn-версии, но на преобразованных признаках почти догоняет её по качеству.



### Сводная таблица по регрессии

| Модель                                   |      MSE |      MAE |   R^2  |
| :--------------------------------------- | -------: | -------: | :---: |
| Sklearn (до улучшения)                   | 2.33e+08 | 10613.91 | 0.903 |
| Sklearn (после улучшения)                | 2.27e+08 | 10488.54 | 0.905 |
| Собственная имплементация (до улучшения) | 2.33e+08 | 10613.91 | 0.903 |
| Собственная имплементация (после улучш.) | 2.32e+08 | 10616.83 | 0.903 |



### Сводная таблица по классификации 

| Модель                                   | Accuracy | Precision | Recall | F1-score |
| :--------------------------------------- | -------: | --------: | -----: | -------: |
| Sklearn (до улучшения)                   |   84.29% |    84.23% | 84.29% |   84.24% |
| Sklearn (после улучшения)                |   84.50% |    84.45% | 84.50% |   84.46% |
| Собственная имплементация (до улучшения) |   76.76% |    80.38% | 76.76% |   76.84% |
| Собственная имплементация (после улучш.) |   82.31% |    82.35% | 82.31% |   82.10% |
